# [SQLite as a Vector Store with SQLiteVec](https://python.langchain.com/docs/integrations/vectorstores/sqlitevec/)

This notebook covers how to get started with the SQLiteVec vector store.

```
**SQLite-Vec** is an SQLite extension designed for vector search, emphasizing local-first operations and easy integration into applications without external servers. It is the successor to **SQLite-VSS** by the same author. It is written in zero-dependency C and designed to be easy to build and use.
```

This notebook shows how to use the SQLiteVec vector database.

In [ ]:
# You need to install sqlite-vec as a dependency.
%pip install --upgrade --quiet  sqlite-vec

In [ ]:
import sqlite3
import sqlite_vec
from langchain_community.vectorstores.sqlitevec import SQLiteVec
from langchain.embeddings import SentenceTransformerEmbeddings

# Initialize your embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create a connection to your SQLite database
db_path = "./tmp/vec.db"
con = sqlite3.connect(db_path)
con.enable_load_extension(True)

# sqlite_vec.load(con)

# Load the sqlite-vec extension
# con.execute("SELECT load_extension('sqlite_vec')")

# Initialize the SQLiteVec instance
vector_store = SQLiteVec(table="state_union", connection=con, embedding=embedding_function)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import SQLiteVec
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("../../../text_files/state_of_the_union.txt")
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]


# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
connection = SQLiteVec.create_connection(db_file="./tmp/vec.db")

db1 = SQLiteVec(
    table="state_union", embedding=embedding_function, connection=connection
)

In [ ]:
for text in texts:
    db1.add_texts([text])

In [ ]:
db1.add_texts(["Ketanji Brown Jackson is awesome"])

In [ ]:
# query it again
query = "What did the president say about Ketanji Brown Jackson"
data = db1.similarity_search(query)

# print results
for row in data:
    print(row.page_content)
    print("=======================")

In [ ]:
# query it again
query = "What did the president say about Today?"
data = db1.similarity_search(query)

# print results
for row in data:
    print(row.page_content)
    print("=======================")